In [0]:
scopes = dbutils.secrets.listScopes()
display(scopes)

In [0]:
def mount_adls(storage_account_name, container_name):
    # Get secrets from Key Vault
    client_id = dbutils.secrets.get(scope = 'demo-secrets-scope', key = 'formula1-app-client-id')
    tenant_id = dbutils.secrets.get(scope = 'demo-secrets-scope', key = 'formula1-app-tenant-id')
    client_secret = dbutils.secrets.get(scope = 'demo-secrets-scope', key = 'formula1-app-client-secret')
    
    # Set spark configurations
    configs = {"fs.azure.account.auth.type": "OAuth",
              "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
              "fs.azure.account.oauth2.client.id": client_id,
              "fs.azure.account.oauth2.client.secret": client_secret,
              "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"}
    
    # Unmount the mount point if it already exists
    if any(mount.mountPoint == f"/mnt/{storage_account_name}/{container_name}" for mount in dbutils.fs.mounts()):
        dbutils.fs.unmount(f"/mnt/{storage_account_name}/{container_name}")
    
    # Mount the storage account container
    dbutils.fs.mount(
      source = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/",
      mount_point = f"/mnt/{storage_account_name}/{container_name}",
      extra_configs = configs)
    
    display(dbutils.fs.mounts())

In [0]:
mount_adls('formula1dl0912', 'raw')

In [0]:
dbutils.fs.mounts()

In [0]:
dbutils.fs.ls('/mnt/formula1dl0912/raw/2021-03-21')

In [0]:
df_circuits = spark.read.csv('/mnt/formula1dl0912/raw/2021-03-21/circuits.csv', header=True, inferSchema=True)
display(df_circuits)